#  <center>Using OpenStreetMap Data to Explore Providence, RI</center>
## <Center>Map Area</Center>

<center><img src="PVD_map.jpg" alt="Drawing" style="width: 400px;"/></center>


Providence, RI, United States

- [https://www.openstreetmap.org/relation/191210](https://www.openstreetmap.org/relation/191210)
- [http://overpass-api.de/query_form.html](http://overpass-api.de/query_form.html)
>**Overpass API > Query**: (node(41.7416, -71.5461, 41.8926, -71.3006);<;); out meta;  

#### Why Providence?
>I am currently searching for a new apartment in Providence, and I have questions about the local amenities available in different neighborhoods. In this project, I will use OpenStreetMap data to address some of my questions. For the benefit of future residents, I will also make suggestions to OpenStreetMap.org to ensure a clear and accurate representation of Providence in the database. 

<p style="page-break-after:always;"></p>

## <center><u>Data Audit</u></center>
After exporting the OpenStreetMap data using the overpass-api query listed above, I began the process of auditing to identity any problems with the data. 

### <Center>Problems with State Names</Center>
#### List of States:  
>RI 557  
Massachusetts 1  
MA 4  
ri 9  
Rhode Island 7

#### Check for Validity 
>As seen in the output copied above, three of the listed state names do not conform to the convention used for states in the OpenStreetMap data (i.e., a capitalized two-letter abbreviation). The three names that do not conform to this convention will be modified during the cleaning phase to better align with the naming convention.

#### Check for Accuracy
> Both Massachusetts and Rhode Island appear in the OSM map of the area surrounding Providence. It therefore makes sense that nodes and ways from both states would appear in the data. No change is required.

### <center>Problems with City Names</center>

#### List of Cities:  
>Providence 204  
Cranston 67  
Central Falls 6  
East Providence 17  
Pawtucket 33  
North Providence 20  
Johnston 31  
Seekonk 9  
cranston 1  
Barrington 100  
Rumford 6  
Smithfield 4  
providence 4  
Warwick 5  
Barrington, Rhode Island 2  
Lincoln 2  

#### Check for Validity
>As seen in the output above, two of the city name values are not capitalized and one of the values for city includes both the city and state. In the data cleaning phase, all words that do not start with capital letters will be modified to begin with capital letters. Additionally, for the entry that includes both the city and state, the state will be removed.

#### Check for Accuracy
>To check for accuracy, I used the code below to ensure that if a city and state were both listed for an entry, the city actually existed within the state (i.e., if the state was Rhode Island, then the city should be a city within Rhode Island). No discrepancies were identified and, thus, no changes were required.

#### Python3 Code
    def state_check(filename):
            osm_file = open(filename, 'r')
            mass_cities = ['Seekonk']
            ri_cities = ['Providence', 'Cranston', 'Central Falls', 'East Providence', 'Pawtucket', 
            'North Providence', 'Johnston', 'cranston', 'Barrington', 'Rumford', 'Smithfield', 'providence',
            'Warwick', 'Barrington, Rhode Island', 'Lincoln'] 
            
            for event, elem in ET.iterparse(osm_file):
                if elem.tag == 'way' or elem.tag == 'node':
                    state = find_state(elem)
                    city = find_city(elem)
                    if city:
                        if state:
                            if(state == 'MA') and (city not in mass_cities):
                                print('Missmatch - The following city is not in Mass: '+city)
                                print(elem.attrib['id'])
                                print(elem.attrib['lat'],elem.attrib['lon'])
                                for tag in elem.iter("tag"):
                                    print(tag.attrib['k'],tag.attrib['v'])
                                print()
                            elif (state == 'RI') and (city not in ri_cities):
                                print('Missmatch - The following city is not in RI:' +city)
                                print(elem.attrib['id'])
                                print(elem.attrib['lat'],elem.attrib['lon'])
                                for tag in elem.iter("tag"):
                                    print(tag.attrib['k'],tag.attrib['v'])
                                print()

    state_check('Providence')       

### <center>Problems with Zipcodes</center>

#### List of Zipcodes:  
>02903 37  
02908 22  
02910 48  
02863 5  
02914 15  
02860 23  
02906 47  
02905 11  
02919 31  
02920 45  
02915 4  
02866 1  
02911 5  
02904 19  
02909 9  
02806 101  
02916 6  
02861 8  
02917 5  
02912 113  
02907 45  
02901 2  
02888 4   
02771 8  
02921 2  
02918 1  
02912 Unset 1  
02906-1189 1  
029212 1  
02903-2996 1  
02865 2  
02906-4800 1  

#### Checking for Validity
>As seen in the output above, three of the zipcodes include additional four-digit delivery sector codes following the primary five-digit zipcode. To conform to the convention used by OpenStreetMap, the data will be cleaned to remove the delivery sector codes and inlcude only the five-digit zipcode. For the "Unset" zipcode, I confirmed its accuracy and will remove the "unset" marker in the cleaning process. 

#### Checking for Accuracy
>I used the code below to check each entry to ensure that, if a city and zipcode were both listed in a given entry, the zipcode actually appeared within the city. To check for this, I generated a dictionary of zipcodes for each city based on the database of zipcodes maintained by UnitedStatesZipcodes.org. In addition to the invalidly coded zipcodes identified above, the program identified two entries with zipcodes that did no appear in the listed city. After reviewing other identifying data for the entry (e.g., street address, location name, latitude and longitude), it became clear that the zipcode was inaccurately listed. During the cleaning process, the inaccurate zipcodes will be replaced by the correct zipcodes for those locations.

#### Python3 Code
    def zipcode_check(filename):
        zipcode_dict = {
            'Seekonk':['02771'],
            'Providence':['02909','02908', '02906','02907','02905','02904','02903','02911','02910',
            '02860','02919','02920','02912','02902','02918','02901','02940'],
            'Cranston':['02920','02910','02921','02905','02831','02907','02909'],
            'Central Falls':['02863'],
            'East Providence':['02914','02915','02916','02860'],
            'Pawtucket':['02860','02861','02863','02862'],
            'North Providence':['02911','02904'],
            'Johnston':['02919'],
            'Barrington':['02806'],
            'Rumford':['02916'],
            'Smithfield':['02917'],
            'Warwick':['02886','02889','02888','02818','02893','02887'],
            'Lincoln':['02865']}
        osm_file = open(filename, 'r')
        n = 1
        for event, elem in ET.iterparse(osm_file):
            if elem.tag == 'way' or elem.tag == 'node':
                zipcode = find_zipcode(elem)
                city = find_city(elem)
                if zipcode:
                    if city:
                        if zipcode not in zipcode_dict[city]:
                            print(n)
                            print('Missmatch: '+ zipcode + " not in " + city)
                            print(elem.attrib['id'])
                            try:
                                print(elem.attrib['lat'],elem.attrib['lon'])
                            except:
                                pass
                            for tag in elem.iter("tag"):
                                print(tag.attrib['k'],tag.attrib['v'])
                            print()
                            n+=1

    zipcode_check('Providence') 

### <center>Problems with Street Names</center>

#### Dictionary of Unexpected Street Suffixes
>'Alley': {'Fones Alley'}  
 'Ave': {'Atwood Ave',
         'Elmwood Ave',
         'Park Ave',
         'Reservoir Ave',
         'Wayland Ave'}  
 'BowenStreet': {'BowenStreet'}  
 'Broadway': {'Broadway'}  
 'Highway': {'Middle Highway'}  
 'Hill': {'Capitol Hill'}  
 'Pike': {'Putnam Pike'}  
 'Plaza': {'Kennedy Plaza', 'Regency Plaza'}  
 'Rd': {'Budlong Rd'}  
 'Sq.': {'Cunningham Sq.'}  
 'St': {'Cushing St', 'Pitman St', 'Cranston St', 'Briggs St'}  
 'Way': {'Recreation Way', 'Iron Horse Way'}  
 'Wy': {'Iron Horse Wy'}
 
#### Check for Validity
>Code (recreated below) was written to test the last word of any street name against a list of common street suffixes. If the last word of a given street name was not included among the list of suffixes, it was added as a key to a dictionary of uncommon street endings. The entire street name was included within a list of values for that key. As you can see from the output above, some street endings are either valid, but uncommon suffixs (e.g., Hill, Highway, Alley) or reflect one word street names (e.g., Broadway). Although these streets do not appear in the list of common suffixes, they are still valid and do not need to be changed. Other entries reflect abbreviations that do not conform to OpenStreetMap conventions (e.g., Wy, St, Sq.) and will be modified during the data cleaning phase. 

#### Check for Accuracy
>A check for accurcy was not conducted. It is beyond the scope of this current project to determine whether each street name that appears in the data is actually a street name that appears in a given city.

#### Python3 Code
    street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
    street_types = defaultdict(set)
    osm_file = open('Providence', 'r')

    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", 
    "Square", "Lane", "Road", "Trail", "Parkway", "Commons"]

    def audit_street_types(street_types, street_name):
        m = street_type_re.search(street_name)
        if m:
            street_type = m.group()
            if street_type not in expected:
                street_types[street_type].add(street_name)

    def is_street_name(elem):
        return (elem.attrib['k'] == "addr:street")

    def audit(filename):
        for event, elem in ET.iterparse(filename, events=("start",)):
            if elem.tag == "way":
                for tag in elem.iter("tag"):
                    if is_street_name(tag):
                        audit_street_types(street_types, tag.attrib['v'])
        pprint.pprint(dict(street_types))

    audit(osm_file)

## <Center><u>Data Cleaning</u></Center>

Following the audit, code (available in "OSM_Cleaning" file) was used to modify the OpenStreetMap data based on the corrections idenfied during the audit phase and then save the cleaned data in a new OSM file called ("Providence_Clean"). 

## <Center><u>Data Transformation</u></Center>

Additional code (available in "OSM_Creating_CSV" file) was used to transform the XML data contained within the cleaned Providence OSM file into .csv files that could be easily imported as tables in SQL. The new files are structured to match the scemas that will be used in the SQL tables. 

## <center><u>Creating the SQL Tables</u></center>

The following code was used to create SQL tables. 

    create table pvd_nodes(ID integer [PRIMARY KEY], Lat numeric, Lon numeric, 
    User text, UserID integer, Version integer, ChangeSet integer, Timestamp timestamp) ;
    .mode csv 
    .import PVD_nodes.csv pvd_nodes 

    create table pvd_nodes_tags(ID integer [PRIMARY KEY], Key text, Value text, Type text) ;
    .mode csv 
    .import PVD_nodes_tags.csv pvd_nodes_tags 

    create table pvd_ways(ID integer [PRIMARY KEY], User text, UserID integer, 
    Version integer, ChangeSet integer, Timestamp timestamp) ;
    .mode csv 
    .import PVD_ways.csv pvd_ways 

    create table pvd_ways_nodes(ID integer [PRIMARY KEY], Node_ID integer, Position integer) ;
    .mode csv 
    .import PVD_ways_nodes.csv pvd_ways_nodes 

    create table pvd_ways_tags(ID integer [PRIMARY KEY], Key text, Value text, Type text) ;
    .mode csv 
    .import PVD_ways_tags.csv pvd_ways_tags 
    ![image.png](attachment:image.png)

## <center><u>About the Data</u></center>

### File Sizes
>Providence_Clean.osm .......71.8 MB  
pvd.db..................................38.5 MB  
PVD_nodes.csv....................25.7 MB  
PVD_nodes_tags.csv.............1.4 MB  
PVD_ways.csv........................2.7 MB  
PVD_ways_nodes.csv............8.7 MB  
PVD_ways_tags.csv...............4.7 MB

#### Python3 Code
    def convert_bytes(num):
        for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
            if num < 1024.0:
                return "%3.1f %s" % (num, x)
            num /= 1024.0

    def file_size(file_path):
        if os.path.isfile(file_path):
            file_info = os.stat(file_path)
            return convert_bytes(file_info.st_size)

### Number of Nodes
313,591

    sqlite3> SELECT COUNT(*) FROM pvd_nodes;

### Number of Ways
46,648

    sqlite3> SELECT COUNT(*) FROM pvd_ways;

### Number of Unique Contributers
450

    sqlite3 >
    SELECT COUNT(DISTINCT(all_users.UserID))          
    FROM (SELECT UserID FROM pvd_nodes UNION ALL SELECT UserID FROM pvd_ways)  as all_users;

    
### Top Contributers

greggerm|108876  
woodpeck_fixbot|63399  
jremillard-massgis|57303  
Zirnch|29503  
John Wrenn|9266
Alex KG Ellis|7749  
jerryam|7352  
MassGIS Import|6313  
bot-mode|6306  
EmilyPPP|4685

    sqlite3 > 
    SELECT all_users.User, COUNT(*) as num
    FROM (SELECT User FROM pvd_nodes UNION ALL SELECT User FROM pvd_ways) as all_users
    GROUP BY all_users.user
    ORDER BY num DESC
    LIMIT 10;
    


### Number of Contributions by Year

2007|6,320  
2008|447  
2009|69,339  
2010|801  
2011|44,458  
2012|47,189  
2013|86,395  
2014|16,925  
2015|20,414  
2016|31,554  
2017|36,397  

    sqlite3 >  
    SELECT strftime('%Y', all_entries.timestamp) as year, COUNT(*)
    FROM (SELECT ID, Timestamp FROM pvd_nodes UNION ALL SELECT ID, Timestamp FROM pvd_ways) as all_entries
    GROUP BY year;

### Recommendations for Improving the Data

Given the fluctuations in contributions over the 11-year period in which users made changes to the OpenStreetMap data for Providence RI, it would be interesting to identify any events that may have prompted increases or decreases in the number of contributions for a given year. Specifically, given the strikingly low number of contributions made in 2008 an 2010, I am curious if anything happened during those years that discouraged or prevented changes to the data. Similarly, I want to know what may have led to higher numbers of contributions in 2009 and 2013. Having an understanding of contextual and historical factors that correspond to fluctuations in contributor activity can provide insight into what might be done to promote future contributions. 

## <Center><u>Exploratory Analyses</u></Center>

My exploratory analyses are guided by a desire to decide between two areas to which I am searching for apartments. I would like to compare the different amentities available in each neighborhood. 

### Neighborhood Comparisons

The two neighborhoods to which I am considering moving are College Hill/East Side of Providence and Federal Hill/Downtown Providence. To facilitate my exploration of these neighborhoods, I first had to identify their boundaries. I superimposed a neighborhood map over the map used by OpenStreetMap, and marked rectangles over the neighboorhoods of interest. I used rectangles because this allowed me to specifcy latitude and longitude ranges 33containing each area (which could be used in SQL queries).  

<img src="PVD_Neighborhood_Map_v2.jpg" alt="Drawing" style="width: 600px;"/>

<Center><b>Map of Providence Neighborhood</b></Center>  
<Center>Boxes 1 and 2 cover College Hill/East Side and Box 3 covers Federal Hill/Downtown Providence</Center>



#### Top Five Amenities in Each Neighborhood
Hoping to get a sense of the flavor of both neighborhoods, I initially decided to search to see what the top five amenities were in each.

>College Hill/East Providence
>>place_of_worship,28  
school,25  
bench,23  
restaurant,20  
cafe,18  

>Federal Hill/Downtown Providence
>>bench,34  
waste_basket,16  
cafe,11  
social_facility,10  
place_of_worship,8  

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Key = "amenity") and 
    (((pvd_nodes.Lon >= -71.41) and (pvd_nodes.Lon <= -71.38) and 
    (pvd_nodes.Lat >= 41.825) and (pvd_nodes.Lat <= 41.857)) or 
    ((pvd_nodes.Lon >= -71.407) and (pvd_nodes.Lon <= -71.385) and 
    (pvd_nodes.Lat >= 41.818) and (pvd_nodes.Lat <= 41.825)))
    group by pvd_nodes_tags.value
    order by num desc 
    limit 5;

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Key = "amenity") and 
    (pvd_nodes.Lon <= -71.36) and 
    (pvd_nodes.Lon >= -71.407) and 
    (pvd_nodes.Lat >= 41.813) and 
    (pvd_nodes.Lat <= 41.825)
    group by pvd_nodes_tags.value
    order by num desc 
    limit 5;
    
#### Number of Dry Cleaners in Each Neighborhood
In the past, I have made the mistake of not looking to see if there are local dry cleaners within walking distance of prospective apartments. For the current search, I want to determine the availability of dry cleaners within each neighborhood. 

>College Hill/East Providence
>> 0

>Federal Hill/Downtown Providence
>> 0

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Value = "dry_cleaning") and 
    (((pvd_nodes.Lon >= -71.41) and (pvd_nodes.Lon <= -71.38) and 
    (pvd_nodes.Lat >= 41.825) and (pvd_nodes.Lat <= 41.857)) or 
    ((pvd_nodes.Lon >= -71.407) and (pvd_nodes.Lon <= -71.385) and 
    (pvd_nodes.Lat >= 41.818) and (pvd_nodes.Lat <= 41.825)));
    
    sqlite3 > 
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Value = "dry_cleaning") and 
    (pvd_nodes.Lon <= -71.36) and 
    (pvd_nodes.Lon >= -71.407) and 
    (pvd_nodes.Lat >= 41.813) and 
    (pvd_nodes.Lat <= 41.825);

#### Number of Libraries
As I spend a lot of time completing projects independenlty outside of an office, I wanted to determine if there quite spaces where I might be able to work rather than in my apartment.

>College Hill/East Providence
>> 8 

>Federal Hill/Downtown Providence
>> 2 

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and
    (pvd_nodes_tags.Value = "library") and 
    (((pvd_nodes.Lon >= -71.41) and (pvd_nodes.Lon <= -71.38) and 
    (pvd_nodes.Lat >= 41.825) and (pvd_nodes.Lat <= 41.857)) or 
    ((pvd_nodes.Lon >= -71.407) and (pvd_nodes.Lon <= -71.385) and 
    (pvd_nodes.Lat >= 41.818) and (pvd_nodes.Lat <= 41.825)));
    
    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Value = "library") and 
    (pvd_nodes.Lon <= -71.36) and 
    (pvd_nodes.Lon >= -71.407) and 
    (pvd_nodes.Lat >= 41.813) and 
    (pvd_nodes.Lat <= 41.825);
   


#### Number of Trees
Another important factor in deciding where to move is the amount of local green spaces. I'd like to feel as though I have some access to "nature" within walking distance of my apartment.

>College Hill/East Providence
>>206

>Federal Hill/Downtown Providence
>>256

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Value = "tree") and 
    (((pvd_nodes.Lon >= -71.41) and (pvd_nodes.Lon <= -71.38) and 
    (pvd_nodes.Lat >= 41.825) and (pvd_nodes.Lat <= 41.857)) or 
    ((pvd_nodes.Lon >= -71.407) and (pvd_nodes.Lon <= -71.385) and 
    (pvd_nodes.Lat >= 41.818) and (pvd_nodes.Lat <= 41.825)));

    sqlite3 >
    SELECT pvd_nodes_tags.Value, count(*) as num 
    FROM pvd_nodes_tags, pvd_nodes 
    WHERE (pvd_nodes.ID = pvd_nodes_tags.ID) and 
    (pvd_nodes_tags.Value = "tree") and 
    (pvd_nodes.Lon <= -71.36) and 
    (pvd_nodes.Lon >= -71.407) and 
    (pvd_nodes.Lat >= 41.813) and 
    (pvd_nodes.Lat <= 41.825);

### Additional Recommendations for Improving the Data
To facilitate a more accurate and user-informed exploration of the different neighborhoods around Providence, I would suggest adding "neighborhood" tags to the nodes and ways included in this data. This would great facilitate the ease at which one is able to search for different amenities by neighborhood within a larger metropolitan area.

It would also be helpful to encourage contributors to include more data related to services available at different amenities. Notably, more data related to the location and hours of operations of dry cleaners would also be useful. It appears that the number of dry cleaners is underreported in the data. Given that the "shop" key may be underutilized as a means of identifying dry cleaning services, it may be helpful start including dry cleaner as a value within the more popular "amenitiy" key. 